In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [2]:
base = pd.read_csv('../data/clean/pokemon_withtypes_reference.csv')
more = pd.read_csv('../data/raw/datasets_2756_4568_pokemon.csv')

In [3]:
more.columns

Index(['abilities', 'against_bug', 'against_dark', 'against_dragon',
       'against_electric', 'against_fairy', 'against_fight', 'against_fire',
       'against_flying', 'against_ghost', 'against_grass', 'against_ground',
       'against_ice', 'against_normal', 'against_poison', 'against_psychic',
       'against_rock', 'against_steel', 'against_water', 'attack',
       'base_egg_steps', 'base_happiness', 'base_total', 'capture_rate',
       'classfication', 'defense', 'experience_growth', 'height_m', 'hp',
       'japanese_name', 'name', 'percentage_male', 'pokedex_number',
       'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'weight_kg',
       'generation', 'is_legendary'],
      dtype='object')

In [15]:
more = more[['name', 'hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']]

In [17]:
full_df = base.merge(more, on='name', how='outer')

In [18]:
full_df = full_df[~full_df.id.isna()]

In [19]:
full_df

,id,name,type_1,type_2,hp,attack,defense,sp_attack,sp_defense,speed
0,0.0,Gliscor,Ground,Flying,75.0,95.0,125.0,45.0,75.0,95.0
1,1.0,Fletchinder,Fire,Flying,62.0,73.0,55.0,56.0,52.0,84.0
2,2.0,Poliwrath,Water,Fighting,90.0,95.0,95.0,70.0,90.0,70.0
3,3.0,Cresselia,Psychic,None,120.0,70.0,120.0,75.0,130.0,85.0
4,4.0,Frogadier,Water,None,54.0,63.0,52.0,83.0,56.0,97.0
...,...,...,...,...,...,...,...,...,...,...
708,708.0,Urshifu-Rapid-Strike,Fighting,Water,NaN,NaN,NaN,NaN,NaN,NaN
709,709.0,Slowbro-Galar,Poison,Psychic,NaN,NaN,NaN,NaN,NaN,NaN
710,710.0,Fraxure,Dragon,None,66.0,117.0,70.0,40.0,50.0,67.0
711,711.0,Thwackey,Grass,None,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
problem_set = full_df.loc[~(full_df.hp.isna())]

In [9]:
problem_set

,id,name,type_1,type_2,hp,attack,defense,sp_attack,sp_defense,speed,type1,type2
81,81.0,Rotom-Frost,Electric,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,97.0,Kyurem-Black,Dragon,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,101.0,Rotom-Wash,Electric,Water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,142.0,Thundurus-Therian,Electric,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,151.0,Landorus-Therian,Ground,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
707,707.0,Urshifu,Fighting,Dark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,708.0,Urshifu-Rapid-Strike,Fighting,Water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
709,709.0,Slowbro-Galar,Poison,Psychic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711,711.0,Thwackey,Grass,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
base_url = 'https://serebii.net/pokedex-swsh/'
labels = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']
 
for poke in problem_set.name:
    try:
        url = base_url + poke.lower().split('-')[0] + '/'
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html')
        dextable = soup.find_all('table', {'class':'dextable'})
        base_stats = [t.text for t in dextable if 'Base Stats - Total' in t.text]
        stats = [int(stat) for stat in base_stats[0].split('Base Stats - Total: ')[1].split('\n')[1:7]]
        for label, stat in zip(labels, stats): 
            full_df[label].loc[full_df.name==poke] = stat
    except:
        print (f'{poke} FAILED!')

/Library/Python/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Thundurus-Therian FAILED!
Landorus-Therian FAILED!
Hoopa-Unbound FAILED!
Tornadus-Therian FAILED!
Sceptile-Mega FAILED!
Houndoom-Mega FAILED!
Latios-Mega FAILED!
Altaria-Mega FAILED!
Swampert-Mega FAILED!
Metagross-Mega FAILED!
Latias-Mega FAILED!
Pidgeot-Mega FAILED!
Medicham-Mega FAILED!
Aerodactyl-Mega FAILED!
Aggron-Mega FAILED!
Beedrill-Mega FAILED!
Garchomp-Mega FAILED!
Banette-Mega FAILED!
Diancie-Mega FAILED!
Absol-Mega FAILED!
Ampharos-Mega FAILED!
Camerupt-Mega FAILED!
Audino-Mega FAILED!
Wormadam-Sandy FAILED!
Wormadam-Trash FAILED!
Greninja-Ash FAILED!
Muk-Alola FAILED!
Oricorio-Pom-Pom FAILED!
Oricorio-Sensu FAILED!
Golem-Alola FAILED!
Zygarde-10% FAILED!
Raticate-Alola FAILED!
Rattata-Alola FAILED!
Naganadel FAILED!
Kommo-o-Totem FAILED!
Blacephalon FAILED!
Stakataka FAILED!
Poipole FAILED!
Mr. Mime-Galar FAILED!
Mr. Rime FAILED!


In [23]:
full_df.to_csv('../data/clean/pokemon_withtypes_reference.csv', index=False)